In [25]:
import pandas as pd
import os
#import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.dates import MonthLocator, DateFormatter


from sqlalchemy import create_engine, text

#maximum number of rows to display
pd.options.display.max_rows = 50

DB_USERNAME = 'developers'
DB_PASSWORD = '12345678'
DB_INSTANCE = 'osomaloso'

#DB_INSTANCE = 'localhost'
DB_PORT = '5432'
DB_DATA_BASE = 'cnc'

engine=create_engine(f'postgresql://{DB_USERNAME}:{DB_PASSWORD}@{DB_INSTANCE}:{DB_PORT}/{DB_DATA_BASE}')

def runQuery(sql):
    result = engine.connect().execution_options(isolation_level="AUTOCOMMIT").execute((text(sql)))
    return pd.DataFrame(result.fetchall(), columns=result.keys())


query_03= """
select codenc, telelink, fecha, duracion, edad,  "ciudad", "estrato", edcc.etiqueta as "genero" from (
select codenc, telelink, fecha, duracion, edad, "cod_ciudad", "ciudad", estrato as "cod_estrato", 
edcc.etiqueta as "estrato", genero from(
select codenc, telelink, fecha, duracion, edad, ciudad as "cod_ciudad", edcc.etiqueta as "ciudad", estrato, genero
from ecar_dwh.ecar_dwh_base_encuestas_efectivas edbee 
inner join ecar_dwh.ecar_dwh_base_correspondencia_codigos edcc on edbee.ciudad=edcc.codigo
where edcc.campo='ciudad')tt
inner join ecar_dwh.ecar_dwh_base_correspondencia_codigos edcc on tt.estrato=edcc.codigo
where edcc.campo='estrato')tt1
inner join ecar_dwh.ecar_dwh_base_correspondencia_codigos edcc on tt1.genero=edcc.codigo
where edcc.campo='genero'
"""
df_efectivas = runQuery(query_03)



    

In [19]:
df_efectivas_cuotas=pd.DataFrame()
for i in ['estrato', 'genero']:
    df_count=df_efectivas.groupby(i)[['codenc']].count().rename(columns={'codenc':"count"})
    df_count['variable']=i
    df_efectivas_cuotas=pd.concat([df_count, df_efectivas_cuotas])

In [21]:
df_efectivas_cuotas=df_efectivas_cuotas.sort_values('count',ascending=True)

In [24]:
df_efectivas_cuotas

,count,variable
Alto,10274,estrato
Medio Alto,13370,estrato
Medio Medio,17365,estrato
Medio Bajo,36208,estrato
Bajo,65044,estrato
Hombre,70406,genero
Mujer,71855,genero


In [27]:

query_02=""" select * from ecar_dwh.ecar_dwh_base_modelo """
df_modelo=runQuery(query_02)

cols=['calls', 'total', 'espera', 'hablado', 'disponible', 'pausas',
       'muerto', 'duracion_efectivas','efectividad_ajustada']

df_tiempos=df_modelo[cols]

In [28]:
df_tiempos

,calls,total,espera,hablado,disponible,pausas,muerto,duracion_efectivas,efectividad_ajustada
0,6252.705882,623484.000000,150572.000000,313690.000000,42598.000000,116624.000000,2193.000000,1102.323944,1804.235294
1,10989.000000,147804.000000,40859.500000,61113.500000,14951.500000,30879.500000,289.000000,954.035714,1188.000000
2,6900.927106,820764.181818,235728.090909,330865.454545,88244.272727,165926.363636,7262.818182,739.523805,2980.843057
3,6440.057143,510847.500000,180869.500000,202350.000000,30753.500000,96874.500000,5939.500000,749.815857,1627.714286
4,6828.463768,613878.000000,160398.250000,247391.750000,64383.000000,141705.000000,24955.500000,851.769881,1814.086957
...,...,...,...,...,...,...,...,...,...
167,13697.454545,423112.333333,141536.000000,167878.666667,36364.666667,77333.000000,6066.000000,905.159907,2184.000000
168,6037.558442,549809.500000,216517.500000,199659.000000,31974.500000,101658.500000,6866.000000,764.298469,1826.181818
169,7177.520000,760716.000000,226663.000000,311614.333333,82926.000000,139512.666667,4070.333333,719.768171,3102.240000
170,6085.707762,501703.800000,156219.600000,174043.800000,79735.400000,91705.000000,5142.000000,818.476296,2383.644187


,codenc,efectividad_ajustada,meses_trabajados,como_conocio_cnc,localidad,educacion_formal,estado_educacion,nombre_educacion,porc_efectivas_mujer,porc_efectivas_hombre,...,porc_efectivas_cali,porc_efectivas_bucaramanga,porc_efectivas_barranquilla,porc_efectivas_medellín,porc_efectivas_bogotá,porc_efectivas_rango_edad_5,porc_efectivas_rango_edad_2,porc_efectivas_rango_edad_4,porc_efectivas_rango_edad_3,porc_efectivas_rango_edad_1
0,Encuestador 1,1804.235294,1,internet - computrabajo.com,otros,profesional,aplazado,otros,58,42,...,6,4,3,24,30,23,15,15,20,27
1,Encuestador 10,1188.000000,2,voz a voz- referido por un conocido,otros,no_disponible,no_disponible,no_disponible,18,18,...,0,0,0,27,0,18,9,0,0,9
2,Encuestador 100,2980.843057,11,internet - zonajobs.com,otros,no_disponible,no_disponible,no_disponible,46,47,...,11,5,10,8,29,14,14,17,21,27
3,Encuestador 102,1627.714286,2,internet - computrabajo.com,centro oriente,tecnólogo,no_disponible,ciencias ambientales y salud,51,49,...,11,2,7,14,49,20,24,17,18,22
4,Encuestador 104,1814.086957,4,voz a voz- referido por un conocido,centro oriente,bachiller,culminado,no_disponible,38,34,...,9,7,5,11,27,17,11,12,16,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,Encuestador 91,2184.000000,3,voz a voz- referido por un conocido,otros,bachiller,culminado,no_disponible,53,47,...,27,1,6,4,21,16,16,20,23,25
168,Encuestador 93,1826.181818,2,otros,sur_occidente,bachiller,culminado,no_disponible,62,38,...,11,3,10,10,44,28,10,16,20,25
169,Encuestador 95,3102.240000,3,otros,sur_occidente,bachiller,culminado,no_disponible,53,47,...,8,5,12,9,25,19,17,17,18,29
170,Encuestador 96,2383.644187,5,voz a voz- referido por un conocido,otros,no_disponible,no_disponible,no_disponible,45,55,...,7,8,12,14,26,12,17,16,19,36


In [66]:
df_modelo.info()
df_modelo.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Columns: 115 entries, codenc to %efectivas_rango_edad_1
dtypes: float64(48), int32(60), int64(1), object(6)
memory usage: 114.3+ KB


Index(['codenc', 'calls', 'total', 'espera', 'hablado', 'disponible', 'pausas',
       'muerto', 'duracion_efectivas', 'dias_trabajados',
       ...
       '%efectivas_cali', '%efectivas_bucaramanga', '%efectivas_barranquilla',
       '%efectivas_medellín', '%efectivas_bogotá', '%efectivas_rango_edad_5',
       '%efectivas_rango_edad_2', '%efectivas_rango_edad_4',
       '%efectivas_rango_edad_3', '%efectivas_rango_edad_1'],
      dtype='object', length=115)

In [76]:
cols=['efectivas_mujer', 'efectivas_hombre','efectivas_alto', 'efectivas_medio_alto', 'efectivas_medio_medio','efectivas_bajo', 'efectivas_medio_bajo', 'efectivas_tunja',
       'efectivas_popayán', 'efectivas_montería', 'efectivas_manizales','efectivas_armenia', 'efectivas_villavicencio', 'efectivas_pasto','efectivas_cúcuta', 'efectivas_santa_marta', 'efectivas_ibagué',
       'efectivas_neiva', 'efectivas_pereira', 'efectivas_cartagena','efectivas_cali', 'efectivas_bucaramanga', 'efectivas_barranquilla','efectivas_medellín', 'efectivas_bogotá', 'efectivas_rango_edad_5',
       'efectivas_rango_edad_2', 'efectivas_rango_edad_4','efectivas_rango_edad_3', 'efectivas_rango_edad_1']
for i in cols:
    df_modelo["%" + str(i)]=round((df_modelo[i]/df_modelo['efectividad_ajustada'])*100).astype(int)
df_modelo[['codenc','efectividad_ajustada','meses_trabajados','como_conocio_cnc','localidad','educacion_formal','estado_educacion','nombre_educacion',
         '%efectivas_mujer', '%efectivas_hombre', '%efectivas_alto','%efectivas_medio_alto', '%efectivas_medio_medio',
         '%efectivas_bajo', '%efectivas_medio_bajo','%efectivas_tunja', '%efectivas_popayán','%efectivas_montería', 
         '%efectivas_manizales','%efectivas_armenia', '%efectivas_villavicencio','%efectivas_pasto', '%efectivas_cúcuta',
         '%efectivas_santa_marta', '%efectivas_ibagué','%efectivas_neiva', '%efectivas_pereira','%efectivas_cartagena', '%efectivas_cali',
         '%efectivas_bucaramanga', '%efectivas_barranquilla','%efectivas_medellín', '%efectivas_bogotá','%efectivas_rango_edad_5', '%efectivas_rango_edad_2',
         '%efectivas_rango_edad_4', '%efectivas_rango_edad_3','%efectivas_rango_edad_1']]

,codenc,efectividad_ajustada,meses_trabajados,como_conocio_cnc,localidad,educacion_formal,estado_educacion,nombre_educacion,%efectivas_mujer,%efectivas_hombre,...,%efectivas_cali,%efectivas_bucaramanga,%efectivas_barranquilla,%efectivas_medellín,%efectivas_bogotá,%efectivas_rango_edad_5,%efectivas_rango_edad_2,%efectivas_rango_edad_4,%efectivas_rango_edad_3,%efectivas_rango_edad_1
0,Encuestador 1,1804.235294,1,internet - computrabajo.com,otros,profesional,aplazado,otros,58,42,...,6,4,3,24,30,23,15,15,20,27
1,Encuestador 10,1188.000000,2,voz a voz- referido por un conocido,otros,no_disponible,no_disponible,no_disponible,18,18,...,0,0,0,27,0,18,9,0,0,9
2,Encuestador 100,2980.843057,11,internet - zonajobs.com,otros,no_disponible,no_disponible,no_disponible,46,47,...,11,5,10,8,29,14,14,17,21,27
3,Encuestador 102,1627.714286,2,internet - computrabajo.com,centro oriente,tecnólogo,no_disponible,ciencias ambientales y salud,51,49,...,11,2,7,14,49,20,24,17,18,22
4,Encuestador 104,1814.086957,4,voz a voz- referido por un conocido,centro oriente,bachiller,culminado,no_disponible,38,34,...,9,7,5,11,27,17,11,12,16,18
5,Encuestador 105,1982.415669,8,internet - computrabajo.com,norte,técnico,actualmente,ciencias ambientales y salud,53,47,...,11,5,7,16,30,19,16,22,20,23
6,Encuestador 107,2144.752941,5,internet - pagina web del cnc,norte,bachiller,culminado,no_disponible,48,52,...,7,2,27,8,29,25,23,10,21,21
7,Encuestador 11,432.000000,1,voz a voz- referido por un conocido,otros,no_disponible,no_disponible,no_disponible,0,100,...,0,0,0,0,100,100,0,0,0,0
8,Encuestador 112,2277.000000,4,voz a voz- referido por un conocido,norte,técnico,aplazado,ciencias administrativas y contables,34,41,...,6,4,5,10,30,17,7,12,17,21
9,Encuestador 113,1528.615385,2,internet - computrabajo.com,norte,técnico,culminado,ciencias ambientales y salud,48,52,...,9,7,13,13,35,20,9,20,17,35


In [75]:

pd.options.display.max_rows =200
df_modelo.columns

Index(['codenc', 'calls', 'total', 'espera', 'hablado', 'disponible', 'pausas',
       'muerto', 'duracion_efectivas', 'dias_trabajados',
       ...
       '%efectivas_cali', '%efectivas_bucaramanga', '%efectivas_barranquilla',
       '%efectivas_medellín', '%efectivas_bogotá', '%efectivas_rango_edad_5',
       '%efectivas_rango_edad_2', '%efectivas_rango_edad_4',
       '%efectivas_rango_edad_3', '%efectivas_rango_edad_1'],
      dtype='object', length=115)

In [48]:
df_efectivas[df_efectivas['codenc']=='Encuestador 10']

,codenc,telelink,fecha,duracion,edad,ciudad,estrato,genero
37727,Encuestador 10,210910.0,2018-04-29,1025,46,Medellín,Bajo,Hombre
37743,Encuestador 10,210942.0,2018-04-29,1109,49,Medellín,Medio Bajo,Hombre
37832,Encuestador 10,211098.0,2018-04-30,916,19,Medellín,Bajo,Mujer
37864,Encuestador 10,211149.0,2018-04-30,492,55,Cartagena,Bajo,Mujer
127008,Encuestador 10,NaN,2018-05-04,876,31,Bogotá,Medio Medio,Mujer
127043,Encuestador 10,NaN,2018-05-04,638,57,Cali,Bajo,Hombre
127141,Encuestador 10,NaN,2018-05-05,969,50,Bogotá,Medio Bajo,Hombre
127153,Encuestador 10,NaN,2018-05-05,1450,55,Barranquilla,Medio Bajo,Mujer
127209,Encuestador 10,NaN,2018-05-05,1644,50,Cúcuta,Medio Bajo,Mujer
127230,Encuestador 10,NaN,2018-05-05,905,58,Manizales,Medio Bajo,Hombre


In [77]:
df_modelo.sort_values('efectividad_ajustada',ascending=False)

,codenc,calls,total,espera,hablado,disponible,pausas,muerto,duracion_efectivas,dias_trabajados,...,%efectivas_cali,%efectivas_bucaramanga,%efectivas_barranquilla,%efectivas_medellín,%efectivas_bogotá,%efectivas_rango_edad_5,%efectivas_rango_edad_2,%efectivas_rango_edad_4,%efectivas_rango_edad_3,%efectivas_rango_edad_1
74,Encuestador 221,12072.000000,59408.000000,15254.000000,27339.000000,4086.000000,12729.000000,175.000000,722.333333,1.000000,...,33,0,0,11,22,0,22,33,22,22
95,Encuestador 250,17544.000000,78041.000000,24160.000000,29769.000000,9992.000000,14120.000000,257.000000,698.333333,1.000000,...,0,22,0,0,33,11,22,33,11,22
23,Encuestador 139,4910.400000,169407.000000,36917.000000,92278.000000,8688.000000,31524.000000,288.000000,787.738095,5.000000,...,24,5,5,10,14,10,14,19,17,40
155,Encuestador 66,2688.000000,31483.000000,7911.000000,13933.000000,4316.000000,5323.000000,33.000000,946.625000,1.000000,...,0,12,0,12,50,25,38,12,12,12
145,Encuestador 43,22920.000000,92687.000000,27817.000000,37255.000000,10313.000000,17302.000000,192.000000,992.750000,1.000000,...,25,0,0,0,38,0,0,12,12,75
117,Encuestador 282,6667.895474,677908.043478,184905.043478,257518.260870,52552.086957,182932.652174,5129.434783,699.288920,19.000000,...,10,5,9,11,27,14,16,17,20,24
40,Encuestador 169,6339.873055,613901.833333,151207.666667,275305.833333,44957.333333,142431.000000,3892.583333,671.185635,18.181818,...,11,6,9,13,30,15,20,16,18,26
64,Encuestador 204,6257.974268,610599.285714,167838.571429,252683.428571,53068.142857,137009.142857,15986.285714,909.151120,15.714286,...,10,9,7,13,22,14,10,16,16,25
86,Encuestador 240,7152.737436,741869.500000,176200.666667,333727.166667,73640.333333,158301.333333,5268.166667,726.689342,19.500000,...,10,4,12,6,28,11,14,16,18,23
123,Encuestador 289,5374.428571,588905.500000,209103.000000,242541.000000,28576.500000,108685.000000,7282.000000,736.293162,15.500000,...,10,2,8,12,35,19,10,17,22,32
